In [1]:
### Code Space for MATH280 Proj. 1 ###

In [2]:
# Import needed python module for file reading
import os

In [3]:
def extract_tones_from_file(file_path):
    """
    Read a text file and extract tones from numbered pinyin syllables.
    Each tone is represented as an integer (1-5) based on numbered pinyin syllables.
    """
    tones = []
    if not os.path.exists(file_path):
        print(f"File {file_path} not found. Please check the file path and try again.")
        return []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line_tones = [int(word[-1]) for word in line.split() if word[-1].isdigit()]
            tones.append(line_tones)
    return tones

## Strategy 1: Log-likelihood Calculation

In [4]:
import sage.all as sa
def construct_markov_matrix(tones_list, num_states=5):
    """
    Create a SageMath Markov matrix for a given list of tone sequences.
    NOTE: The matrix is normalized so that each row sums to 1, which is a bit differ to out definition in the class
    """
    # Initialize with zeros
    transition_counts = sa.Matrix(sa.SR, num_states, num_states, 0)  

    for tones in tones_list:
        for i in range(len(tones) - 1):
            current_tone = tones[i] - 1  # Adjusting to 0-based index
            next_tone = tones[i + 1] - 1  
            transition_counts[current_tone, next_tone] += 1

    for i in range(num_states):
        row_sum = sum(transition_counts[i, j] for j in range(num_states))
        if row_sum > 0:
            for j in range(num_states):
                transition_counts[i, j] /= row_sum  # Normalize the row

    return transition_counts

# Refined Version adding Laplace Smoothing for better performance in prediction
'''
def construct_markov_matrix(tones_list, num_states=5):
    transition_counts = sa.Matrix(sa.SR, num_states, num_states, 1)  # Laplace smoothing (start with 1)

    # Initialize with zeros
    transition_counts = sa.Matrix(sa.SR, num_states, num_states, 0)  

    for tones in tones_list:
        for i in range(len(tones) - 1):
            current_tone = tones[i] - 1  # Adjusting to 0-based index
            next_tone = tones[i + 1] - 1  # Adjusting to 0-based index
            transition_counts[current_tone, next_tone] += 1

    for i in range(num_states):
        row_sum = sum(transition_counts[i, j] for j in range(num_states))
        if row_sum > 0:
            for j in range(num_states):
                transition_counts[i, j] /= row_sum  # Normalize the row

    return transition_counts
'''

'\ndef construct_markov_matrix(tones_list, num_states=5):\n    transition_counts = sa.Matrix(sa.SR, num_states, num_states, 1)  # Laplace smoothing (start with 1)\n\n    # Initialize with zeros\n    transition_counts = sa.Matrix(sa.SR, num_states, num_states, 0)  \n\n    for tones in tones_list:\n        for i in range(len(tones) - 1):\n            current_tone = tones[i] - 1  # Adjusting to 0-based index\n            next_tone = tones[i + 1] - 1  # Adjusting to 0-based index\n            transition_counts[current_tone, next_tone] += 1\n\n    for i in range(num_states):\n        row_sum = sum(transition_counts[i, j] for j in range(num_states))\n        if row_sum > 0:\n            for j in range(num_states):\n                transition_counts[i, j] /= row_sum  # Normalize the row\n\n    return transition_counts\n'

In [5]:
# Strategy 1 - Calculating the Log-liklihood of Markov Matrices

from sage.all import log
def compute_log_likelihood(matrix, test_tones):
    """
    Calculate the log-likelihood of a sequence of tones given a SageMath Markov matrix.
    """
    log_likelihood = 0
    for i in range(len(test_tones) - 1):
        current_state = test_tones[i] - 1
        next_state = test_tones[i + 1] - 1
        probability = matrix[current_state, next_state]
        if probability > 0:
            log_likelihood += log(probability)
        else:
            # Log(0) is -infinity
            log_likelihood += float('-inf')  
    return log_likelihood


In [6]:
def guess_author(test_tones_list, matrix_zhu, matrix_du):
    total_likelihood_zhu = sum(compute_log_likelihood(matrix_zhu, tones) for tones in test_tones_list)
    total_likelihood_du = sum(compute_log_likelihood(matrix_du, tones) for tones in test_tones_list)
    
    print(f"Total log-likelihood for Zhu Shuzhen: {total_likelihood_zhu}")
    print(f"Total log-likelihood for Du Fu: {total_likelihood_du}")

    if total_likelihood_zhu > total_likelihood_du:
        return "Zhu Shuzhen"
    else:
        return "Du Fu"

In [10]:
def display_markov_matrix(matrix):
    """
    Print the Markov matrix in formatted way.
    """
    print("Markov Matrix (5-tone system):")
    print(matrix)

In [22]:
# Test Case:
 
zsz_file = "zsz.txt"
df_file = "df.txt"

zsz_tones = extract_tones_from_file(zsz_file)
df_tones = extract_tones_from_file(df_file)

matrix_zhu = construct_markov_matrix(zsz_tones)
matrix_du = construct_markov_matrix(df_tones)

print("Zhu Shuzhen's Markov Matrix:")
print(matrix_zhu)
print("\nDu Fu's Markov Matrix:")
print(matrix_du)

test_tones_zsz = zsz_tones[:20]  
test_tones_df = df_tones[:20]   

# Predict authorship based on multiple test sequences
print("\nPredicted author for Zhu Shuzhen's test tones: ", guess_author(test_tones_zsz, matrix_zhu, matrix_du))
print("Predicted author for Du Fu's test tones: ", guess_author(test_tones_df, matrix_zhu, matrix_du))

Zhu Shuzhen's Markov Matrix:
[57/200 47/169  29/93  19/62      0]
[53/200   3/13  22/93   7/31      0]
[  7/40 27/169   5/31  13/93      0]
[ 11/40 56/169   9/31 61/186      0]
[     0      0      0      0      0]

Du Fu's Markov Matrix:
[103/314  87/275  46/153  73/242       0]
[ 44/157  73/275  49/153  69/242       0]
[ 27/157  41/275  16/153  49/242       0]
[ 69/314  74/275   14/51  51/242       0]
[      0       0       0       0       0]
Total log-likelihood for Zhu Shuzhen: 6*log(56/169) + 6*log(61/186) + 4*log(29/93) + 12*log(19/62) + 4*log(9/31) + 10*log(57/200) + 8*log(47/169) + 15*log(11/40) + 7*log(53/200) + 3*log(22/93) + 6*log(3/13) + 6*log(7/31) + 3*log(7/40) + 2*log(5/31) + 4*log(27/169) + 6*log(13/93)
Total log-likelihood for Du Fu: 10*log(103/314) + 3*log(49/153) + 8*log(87/275) + 12*log(73/242) + 4*log(46/153) + 6*log(69/242) + 7*log(44/157) + 4*log(14/51) + 6*log(74/275) + 6*log(73/275) + 15*log(69/314) + 6*log(51/242) + 6*log(49/242) + 3*log(27/157) + 4*log(41/275)

In [8]:
# Formal Test Case Implementation


## Strategy 2: Cosine Similarity Calculation

In [13]:
import sage.all as sa

# Markov Matrix Implementation that the SUM of column matrix summed up to 1
def construct_markov_matrix(tones_list, num_states=5):
    transition_counts = sa.Matrix(sa.SR, num_states, num_states) 

    for tones in tones_list:
        for i in range(len(tones) - 1):
            current_tone = tones[i] - 1  # Convert tone to zero-based index
            next_tone = tones[i + 1] - 1
            transition_counts[current_tone, next_tone] += 1

    # Normalize each column to sum to 1
    for j in range(num_states):
        col_sum = sum(transition_counts[i, j] for i in range(num_states))
        if col_sum > 0:
            for i in range(num_states):
                transition_counts[i, j] /= col_sum

    return transition_counts

# Compute the Equilibrium Vector
import numpy as np

def equilibrium_vector(matrix):
    """
    Compute the equilibrium vector using NumPy for numerical stability.
    """
    # Convert SageMath matrix to NumPy array
    matrix_np = np.array(matrix, dtype=float)

    eigenvalues, eigenvectors = np.linalg.eig(matrix_np)

    # Find the eigenvector corresponding to the eigenvalue closest to 1
    idx = np.argmin(np.abs(eigenvalues - 1))

    equilibrium_vec = np.real(eigenvectors[:, idx])
    equilibrium_vec /= equilibrium_vec.sum()

    return equilibrium_vec

In [14]:
import sage.all as sa

# Define Cosine Similarity
def cosine_similarity(vec1, vec2):
    vec1, vec2 = sa.vector(vec1), sa.vector(vec2)
    return vec1.dot_product(vec2) / (vec1.norm() * vec2.norm())

# Define Euclidean Distance
def euclidean_distance(vec1, vec2):
    return (sa.vector(vec1) - sa.vector(vec2)).norm()

# Define Weighted Score Calculation
# FIXME: ABORTED TRY
def weighted_score(cos_sim, dist, alpha=0.5, beta=0.5):
    return alpha * cos_sim + beta * (1 / (1 + dist))

In [15]:
# Author Prediction Calculation
def predict_author(test_vector, vec_zhu, vec_du):
    """
    Predict author using cosine similarity and Euclidean distance
    between the test vector and the poets' equilibrium vectors separately.
    """
    cos_sim_zhu = cosine_similarity(test_vector, vec_zhu)
    cos_sim_du = cosine_similarity(test_vector, vec_du)

    dist_zhu = euclidean_distance(test_vector, vec_zhu)
    dist_du = euclidean_distance(test_vector, vec_du)


    print(f"Cosine Similarity with Zhu Shuzhen: {cos_sim_zhu}")
    print(f"Cosine Similarity with Du Fu: {cos_sim_du}")
    print(f"Euclidean Distance to Zhu Shuzhen: {dist_zhu}")
    print(f"Euclidean Distance to Du Fu: {dist_du}")

    # Predict using both similarity and distance independently
    cos_prediction = "Zhu Shuzhen" if cos_sim_zhu > cos_sim_du else "Du Fu"
    dist_prediction = "Zhu Shuzhen" if dist_zhu < dist_du else "Du Fu"

    return cos_prediction, dist_prediction

In [16]:
# Debug Usage Tool Base 

def debug_matrix(matrix, name="Markov Matrix"):

    print(f"\n{name}:")
    print(matrix)
    print(f"Column sums: {[sum(matrix.column(j)) for j in range(matrix.ncols())]}")
    try:
        eigenvalues = matrix.eigenvalues()
        print(f"Eigenvalues: {eigenvalues}")
    except Exception as e:
        print(f"Error computing eigenvalues: {e}")

In [21]:
# Sample Test Case

zsz_file = "zsz.txt"
df_file = "df.txt"

zsz_tones = extract_tones_from_file(zsz_file)
df_tones = extract_tones_from_file(df_file)

filtered_zsz_tones = [seq for seq in zsz_tones if seq]
filtered_df_tones = [seq for seq in df_tones if seq]

matrix_zhu = construct_markov_matrix(filtered_zsz_tones)
matrix_du = construct_markov_matrix(filtered_df_tones)

debug_matrix(matrix_zhu, "Zhu Shuzhen's Markov Matrix")
debug_matrix(matrix_du, "Du Fu's Markov Matrix")

try:
    vec_zhu = equilibrium_vector(matrix_zhu)
    vec_du = equilibrium_vector(matrix_du)
except ValueError as e:
    print(f"Error computing equilibrium vector: {e}")
    exit()

test_vector_zsz = equilibrium_vector(construct_markov_matrix(zsz_tones[:20]))
test_vector_df = equilibrium_vector(construct_markov_matrix(df_tones[:20]))

print("\nPredicted author for Zhu Shuzhen's test tones:")
cos_pred, dist_pred = predict_author(test_vector_zsz, vec_zhu, vec_du)
print(f"Cosine Similarity Prediction: {cos_pred}")
print(f"Euclidean Distance Prediction: {dist_pred}")

print("\nPredicted author for Du Fu's test tones:")
cos_pred, dist_pred = predict_author(test_vector_df, vec_zhu, vec_du)
print(f"Cosine Similarity Prediction: {cos_pred}")
print(f"Euclidean Distance Prediction: {dist_pred}")

'''
# Paths for input texts
zsz_file = "zsz.txt"
df_file = "df.txt"

# Extract tones from both files
zsz_tones = extract_tones_from_file(zsz_file)
df_tones = extract_tones_from_file(df_file)

filtered_zsz_tones = [seq for seq in zsz_tones if seq]
filtered_df_tones = [seq for seq in df_tones if seq]

matrix_zhu = construct_markov_matrix(filtered_zsz_tones)
matrix_du = construct_markov_matrix(filtered_df_tones)

debug_matrix(matrix_zhu, "Zhu Shuzhen's Markov Matrix")
debug_matrix(matrix_du, "Du Fu's Markov Matrix")

# Compute equilibrium vectors using the NumPy-based method
try:
    vec_zhu = equilibrium_vector(matrix_zhu)
    vec_du = equilibrium_vector(matrix_du)
except ValueError as e:
    print(f"Error computing equilibrium vector: {e}")
    exit()

# Multiple Sequences Prediction
print("\nPredicted author for Zhu Shuzhen's test tones:",
      predict_author(filtered_zsz_tones[:3], vec_zhu, vec_du))
print("Predicted author for Du Fu's test tones:",
      predict_author(filtered_df_tones[:3], vec_zhu, vec_du))

# Single Sequence Prediction
print("\nPredicted author for a single Zhu Shuzhen sequence:",
      predict_author([filtered_zsz_tones[0]], vec_zhu, vec_du))
print("Predicted author for a single Du Fu sequence:",
      predict_author([filtered_df_tones[0]], vec_zhu, vec_du))
'''


Zhu Shuzhen's Markov Matrix:
[57/200 47/169  29/93  19/62      0]
[53/200   3/13  22/93   7/31      0]
[  7/40 27/169   5/31  13/93      0]
[ 11/40 56/169   9/31 61/186      0]
[     0      0      0      0      0]
Column sums: [1, 1, 1, 1, 0]
Eigenvalues: [-1/1450800*(900*sqrt(117109835103492073)*sqrt(130) + 3501358455997)^(1/3)*(I*sqrt(3) + 1) + 41631431/1450800*(-I*sqrt(3) + 1)/(900*sqrt(117109835103492073)*sqrt(130) + 3501358455997)^(1/3) + 1213/725400, -1/1450800*(900*sqrt(117109835103492073)*sqrt(130) + 3501358455997)^(1/3)*(-I*sqrt(3) + 1) + 41631431/1450800*(I*sqrt(3) + 1)/(900*sqrt(117109835103492073)*sqrt(130) + 3501358455997)^(1/3) + 1213/725400, 1/725400*(900*sqrt(117109835103492073)*sqrt(130) + 3501358455997)^(1/3) - 41631431/725400/(900*sqrt(117109835103492073)*sqrt(130) + 3501358455997)^(1/3) + 1213/725400, 1, 0]

Du Fu's Markov Matrix:
[103/314  87/275  46/153  73/242       0]
[ 44/157  73/275  49/153  69/242       0]
[ 27/157  41/275  16/153  49/242       0]
[ 69/314  

'\n# Paths for input texts\nzsz_file = "zsz.txt"\ndf_file = "df.txt"\n\n# Extract tones from both files\nzsz_tones = extract_tones_from_file(zsz_file)\ndf_tones = extract_tones_from_file(df_file)\n\nfiltered_zsz_tones = [seq for seq in zsz_tones if seq]\nfiltered_df_tones = [seq for seq in df_tones if seq]\n\nmatrix_zhu = construct_markov_matrix(filtered_zsz_tones)\nmatrix_du = construct_markov_matrix(filtered_df_tones)\n\ndebug_matrix(matrix_zhu, "Zhu Shuzhen\'s Markov Matrix")\ndebug_matrix(matrix_du, "Du Fu\'s Markov Matrix")\n\n# Compute equilibrium vectors using the NumPy-based method\ntry:\n    vec_zhu = equilibrium_vector(matrix_zhu)\n    vec_du = equilibrium_vector(matrix_du)\nexcept ValueError as e:\n    print(f"Error computing equilibrium vector: {e}")\n    exit()\n\n# Multiple Sequences Prediction\nprint("\nPredicted author for Zhu Shuzhen\'s test tones:",\n      predict_author(filtered_zsz_tones[:3], vec_zhu, vec_du))\nprint("Predicted author for Du Fu\'s test tones:",\n   

In [23]:
print("Equilibrium Vector for Zhu Shuzhen's Matrix:")
print(vec_zhu)
print("Equilibrium Vector for Du Fu's Matrix:")
print(vec_du)

Equilibrium Vector for Zhu Shuzhen's Matrix:
[ 0.29418287  0.24023142  0.15834941  0.3072363  -0.        ]
Equilibrium Vector for Du Fu's Matrix:
[0.31394412 0.28370395 0.16190486 0.24044708 0.        ]


In [24]:
print("Zhu Shuzhen's Markov Matrix:")
print(matrix_zhu)
print("Du Fu's Markov Matrix:")
print(matrix_du)

Zhu Shuzhen's Markov Matrix:
[57/200 47/169  29/93  19/62      0]
[53/200   3/13  22/93   7/31      0]
[  7/40 27/169   5/31  13/93      0]
[ 11/40 56/169   9/31 61/186      0]
[     0      0      0      0      0]
Du Fu's Markov Matrix:
[103/314  87/275  46/153  73/242       0]
[ 44/157  73/275  49/153  69/242       0]
[ 27/157  41/275  16/153  49/242       0]
[ 69/314  74/275   14/51  51/242       0]
[      0       0       0       0       0]


In [20]:
# Formal Test Case Implementaion: 